## Import libraries

In [ ]:
import requests
import re
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

## Unpickle .pkl file back to dataframe

In [ ]:
#Raw data
#data = pd.read_pickle("sources/df_articles.pkl")
#data = pd.read_pickle("sources/df_length_of_articles.pkl")

#Data after preprocessing
data = pd.read_pickle("sources/df_articles_ap.pkl")

data.reset_index(inplace = True, drop = True)
data

## Load konspiratori.sk list


In [ ]:
with open('sources/zoznam_konspiratori.txt') as f:
    lines = f.readlines()
   #print(lines)
    
list_konspiratori = [x.strip() for x in lines] 

## Load mediabiasfactcheck.com list

In [ ]:
with open('sources/zoznam_mediabiasfactcheck.txt') as f:
    lines = f.readlines()
   #print(lines)
    
list_mediabiasfactcheck = [x.strip() for x in lines] 

## Merge of the lists

In [ ]:
list_konspiratori.extend(list_mediabiasfactcheck)
list_konspiratori

## For test

In [ ]:
for i in range(len(data)):
    for website in list_konspiratori:
         domain = re.findall("https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+",data['raw_body'][i])
         print(domain)
        

In [ ]:
txt = data['raw_body'][3]
regex = "HSP"
#txt = re.compile(r'HSP', re.I)

# domains = re.findall(p,txt)
# domains = list(set(domains))

print(txt)
# for domain in domains:
#     print(domain)

text = 'HSP'

if text in txt:
    print("Found!")
else:
    print("Not found!")


## Check usage of non credible sources in article + add new collumn

In [ ]:
new_col_source = []

for article_num in range(len(data)):
    domains = []
    regex = "https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+"
    p = re.compile(regex)
    
    raw_body = data['raw_body'][article_num]
    print(raw_body)
    
    domains = re.findall(p,raw_body)
    domains = list(set(domains))
    print(domains)
    
    #check list if contains URL which consist of source.name, if yes remove them 
    domains = [source for source in domains if not data['source.name'][article_num] in source]
        
    
    print(domains)
    print(data['source.name'][article_num])
    score = 0
    for website in list_konspiratori:
        if score == 1:
            break
        for domain in domains:
            if website in domain:
                score = 1
                break
            else:
                score = 0
    new_col_source.append(score)
    score = 0

data['non_credible_source'] = new_col_source
print(data[['source.name', 'source_name_cred', 'non_credible_source']])

In [ ]:
data[['source.name', 'source_name_cred', 'non_credible_source']].source_name= 'hlavnespravy.sk' &  source_name_cred = 1

In [ ]:
data['non_credible_source'].value_counts()[0]

In [ ]:
data['non_credible_source'].value_counts()[1]

In [ ]:
data['source_name_cred'].value_counts()[0]

In [ ]:
data['source_name_cred'].value_counts()[1]


In [ ]:
data.to_pickle("sources/data_credible_sources.pkl")

In [ ]:
data = pd.read_pickle("sources/data_credible_sources.pkl")
data

In [ ]:
pd.set_option('display.max_rows', None)
data[(data['source.name'] == 'hlavnespravy.sk') & (data['source_name_cred'] == 1)]